In [1]:
import ccdproc
import os

from datetime import date

OBSV_DATE = date(year=2022, month=10, day=27).strftime("%Y-%m-%d")
BASE_DATA_DIR = "/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464"

BIAS_DIR =  os.path.join(BASE_DATA_DIR, "Bias", OBSV_DATE)
DARKS_DIR = os.path.join(BASE_DATA_DIR, "Darks", OBSV_DATE)
FLATS_DIR = os.path.join(BASE_DATA_DIR, "Flats", OBSV_DATE)

DATA_DIRS = [os.path.join(BASE_DATA_DIR, d) for d in os.listdir(BASE_DATA_DIR) if OBSV_DATE in d and os.path.isdir(os.path.join(BASE_DATA_DIR, d))]
CCD_KWARGS = {'unit': 'adu'}
DATA_DIRS

['/run/media/ramon/USBD/Tesis/IturbideDatos/ATOJ339.9469+45.1464/2022-10-27']

In [59]:
# Master Bias

biasImages = ccdproc.ImageFileCollection(BIAS_DIR)
masterBias = ccdproc.Combiner(biasImages.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()
biasFilePath = os.path.join(BIAS_DIR, f"{OBSV_DATE}_MasterBias.fits")
masterBias.write(biasFilePath, overwrite=True)

In [60]:
# Master Dark

rawDarks = ccdproc.ImageFileCollection(DARKS_DIR)
biasCorrectedDir = os.path.join(DARKS_DIR, "bias_corrected")
if not os.path.exists(biasCorrectedDir):
	os.mkdir(biasCorrectedDir)

exposureTime: None
for dark, dark_fname in rawDarks.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	correctedDark = ccdproc.subtract_bias(dark, masterBias)
	exposureTime = correctedDark.header['EXPOSURE']
	correctedDark.write(os.path.join(biasCorrectedDir, f"b{dark_fname}"))

b_darks = ccdproc.ImageFileCollection(biasCorrectedDir)
masterDark = ccdproc.Combiner(b_darks.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()
masterDark.header['EXPOSURE'] = exposureTime
masterDarkFilePath = os.path.join(DARKS_DIR, f"{OBSV_DATE}_MasterDark.fits")
masterDark.write(masterDarkFilePath, overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [61]:
# Master flat

from astropy.units import second

rawFlats = ccdproc.ImageFileCollection(FLATS_DIR)
correctedFlatsDir = os.path.join(FLATS_DIR, "corrected")
if not os.path.exists(correctedFlatsDir):
	os.mkdir(correctedFlatsDir)

for flat, flat_fname in rawFlats.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
	b_flat = ccdproc.subtract_bias(flat, masterBias)
	bd_flat = ccdproc.subtract_dark(b_flat, masterDark, exposure_time='EXPOSURE', exposure_unit=second, scale=True)
	bd_flat.write(os.path.join(correctedFlatsDir, f"bd{flat_fname}"))

bd_flats = ccdproc.ImageFileCollection(correctedFlatsDir)
masterFlat = ccdproc.Combiner(bd_flats.ccds(ccd_kwargs=CCD_KWARGS)).median_combine()
masterFlatFilePath = os.path.join(FLATS_DIR, f"{OBSV_DATE}_MasterFlat.fits")
masterFlat.write(masterFlatFilePath, overwrite=True)

INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader inste

In [77]:
from ccdproc import wcs_project
from astropy.wcs import WCS

from astropy.io import fits

targetWcs: WCS = None
for dataDir in DATA_DIRS:
	shiftedDir = os.path.join(dataDir, "shifted")
	if not os.path.exists(shiftedDir):
		os.mkdir(shiftedDir)

	rawImages = ccdproc.ImageFileCollection(dataDir)
	for img, img_fname in rawImages.ccds(ccd_kwargs=CCD_KWARGS, return_fname=True):
		if not targetWcs:
			# targetWcs = WCS(img.header)
			with fits.open(os.path.join(dataDir, img_fname)) as fitsImg:
				print(fitsImg)
				targetWcs = WCS(fitsImg[1].header)
		else:
			shiftedImg = wcs_project(img, targetWcs)
			shiftedImg.write(os.path.join(shiftedDir, f"s_{img_fname}"), overwrite=True)

AttributeError: 'HDUList' object has no attribute 'summary'

In [ ]:
from analisis.gaia import fotometria

